# Household Impact Analysis

This chapter analyzes how each of the seven Social Security taxation reform options affects individual households across different income levels. The analysis shows both the change in net income and comparative baseline vs. reform scenarios for each policy option.

## Household Example

The household impact analysis uses a representative household with the following characteristics:
- Single elderly tax filer
- Age 70
- Social Security benefits: $30,000 per year
- Employment income varying from $0 to $200,000 in $500 increments
- Located in Florida (no state income tax)
- Standard deduction

For each income level and policy option, we calculate:
1. **Change in Net Income**: The difference in household net income under the reform versus baseline
2. **Baseline vs Reform Comparison**: Total net income under current law versus under each policy reform

In [1]:
# Import necessary libraries
from IPython.display import display, HTML
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
import warnings
warnings.filterwarnings('ignore')

# Define PolicyEngine's color palette
BLACK = "#000000"
BLUE_LIGHT = "#D8E6F3"
BLUE_PRIMARY = "#2C6496"
DARK_BLUE_HOVER = "#1d3e5e"
DARK_GRAY = "#616161"
DARKEST_BLUE = "#0C1A27"
GRAY = "#808080"
LIGHT_GRAY = "#F2F2F2"
MEDIUM_DARK_GRAY = "#D2D2D2"
MEDIUM_LIGHT_GRAY = "#BDBDBD"
WHITE = "#FFFFFF"

print("Libraries loaded successfully!")

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Libraries loaded successfully!


In [2]:
# Define the base household situation for multiple years
def get_household_situation_for_year(year):
    """Create the base household situation for a specific year"""
    return {
        "people": {
            "person1": {
                "age": {str(year): 70 + (year - 2026)},  # Age increases each year
                "social_security_retirement": {str(year): 30000}
            }
        },
        "families": {
            "your family": {"members": ["person1"]}
        },
        "marital_units": {
            "your marital unit": {"members": ["person1"]}
        },
        "tax_units": {
            "your tax unit": {"members": ["person1"]}
        },
        "spm_units": {
            "your household": {"members": ["person1"]}
        },
        "households": {
            "your household": {
                "members": ["person1"],
                "state_name": {str(year): "FL"}
            }
        },
        "axes": [[
            {
                "name": "employment_income",
                "count": 401,
                "min": 0,
                "max": 200000
            }
        ]]
    }

print("Multi-year household situation function defined!")

Multi-year household situation function defined!


In [3]:
# Define all reform functions
def get_option1_reform():
    """Option 1: Full Repeal of Social Security Benefits Taxation"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2026-01-01.2100-12-31": 0
        }
    }, country_id="us")

def get_option2_reform():
    """Option 2: Taxation of 85% of Social Security Benefits"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 0
        }
    }, country_id="us")

def get_option3_reform():
    """Option 3: 85% Taxation with Permanent Senior Deduction Extension"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.contrib.crfb.senior_deduction_extension.applies": {
            "2026-01-01.2100-12-31": True
        }
    }, country_id="us")

def get_option4_reform():
    """Option 4: Social Security Tax Credit System ($500 Credit)"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.contrib.crfb.ss_credit.in_effect": {
            "2026-01-01.2100-12-31": True
        },
        "gov.contrib.crfb.ss_credit.amount.JOINT": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.contrib.crfb.ss_credit.amount.SINGLE": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.contrib.crfb.ss_credit.amount.SEPARATE": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.contrib.crfb.ss_credit.amount.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.contrib.crfb.ss_credit.amount.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.irs.deductions.senior_deduction.amount": {
            "2026-01-01.2100-12-31": 0
        }
    }, country_id="us")

def get_option5_reform():
    """Option 5: Roth-Style Swap"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.contrib.crfb.tax_employer_payroll_tax.in_effect": {
            "2026-01-01.2100-12-31": True
        },
        "gov.contrib.crfb.tax_employer_payroll_tax.percentage": {
            "2026-01-01.2100-12-31": 1.0
        }
    }, country_id="us")

def get_option6_reform():
    """Option 6: Phased Roth-Style Swap"""
    reform_dict = {
        "gov.contrib.crfb.tax_employer_payroll_tax.in_effect": {
            "2026-01-01.2100-12-31": True
        },
        "gov.contrib.crfb.tax_employer_payroll_tax.percentage": {
            "2026-01-01.2026-12-31": 0.1307,
            "2027-01-01.2027-12-31": 0.2614,
            "2028-01-01.2028-12-31": 0.3922,
            "2029-01-01.2029-12-31": 0.5229,
            "2030-01-01.2030-12-31": 0.6536,
            "2031-01-01.2031-12-31": 0.7843,
            "2032-01-01.2032-12-31": 0.9150,
            "2033-01-01.2100-12-31": 1.0
        },
        "gov.irs.social_security.taxability.rate.base": {
            "2029-01-01.2029-12-31": 0.45,
            "2030-01-01.2030-12-31": 0.40,
            "2031-01-01.2031-12-31": 0.35,
            "2032-01-01.2032-12-31": 0.30,
            "2033-01-01.2033-12-31": 0.25,
            "2034-01-01.2034-12-31": 0.20,
            "2035-01-01.2035-12-31": 0.15,
            "2036-01-01.2036-12-31": 0.10,
            "2037-01-01.2037-12-31": 0.05,
            "2038-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2029-01-01.2029-12-31": 0.80,
            "2030-01-01.2030-12-31": 0.75,
            "2031-01-01.2031-12-31": 0.70,
            "2032-01-01.2032-12-31": 0.65,
            "2033-01-01.2033-12-31": 0.60,
            "2034-01-01.2034-12-31": 0.55,
            "2035-01-01.2035-12-31": 0.50,
            "2036-01-01.2036-12-31": 0.45,
            "2037-01-01.2037-12-31": 0.40,
            "2038-01-01.2038-12-31": 0.35,
            "2039-01-01.2039-12-31": 0.30,
            "2040-01-01.2040-12-31": 0.25,
            "2041-01-01.2041-12-31": 0.20,
            "2042-01-01.2042-12-31": 0.15,
            "2043-01-01.2043-12-31": 0.10,
            "2044-01-01.2044-12-31": 0.05,
            "2045-01-01.2100-12-31": 0
        }
    }
    return Reform.from_dict(reform_dict, country_id="us")

def get_option7_reform():
    """Option 7: Eliminate Bonus Senior Deduction

    Eliminates the $6,000 bonus senior deduction from the One Big Beautiful Bill
    that has a 6% phase-out beginning at $75k/$150k for single/joint filers.
    The deduction expires in 2029, so there's only impact from 2026-2028.
    """
    return Reform.from_dict({
        "gov.irs.deductions.senior_deduction.amount": {
            "2026-01-01.2100-12-31": 0
        }
    }, country_id="us")

print("Reform functions defined successfully!")

Reform functions defined successfully!


In [4]:
# Function to calculate impacts for all years and create a combined dataframe
def calculate_all_years_impact(reform_func, reform_name):
    """Calculate impacts for all years 2026-2035 and return a DataFrame"""
    all_data = []
    years = range(2026, 2036)
    
    for year in years:
        print(f"  Calculating {reform_name} for {year}...")
        situation = get_household_situation_for_year(year)
        reform = reform_func()
        
        # Calculate reform net income
        simulation_reform = Simulation(
            reform=reform,
            situation=situation,
        )
        reform_net_income = simulation_reform.calculate("household_net_income", year)
        
        # Calculate baseline net income
        simulation_baseline = Simulation(
            situation=situation,
        )
        baseline_net_income = simulation_baseline.calculate("household_net_income", year)
        
        # Calculate change in net income
        change_in_net_income = reform_net_income - baseline_net_income
        
        # Add data for this year
        employment_income = [i * 500 for i in range(401)]
        for i, emp_income in enumerate(employment_income):
            all_data.append({
                'reform': reform_name,
                'year': year,
                'employment_income': emp_income,
                'baseline_net_income': baseline_net_income[i],
                'reform_net_income': reform_net_income[i],
                'change_in_net_income': change_in_net_income[i]
            })
    
    return pd.DataFrame(all_data)

# Function to create animated change graph
def create_animated_change_graph(df, reform_name):
    """Create an animated graph showing change in net income across years"""
    
    fig = px.line(
        df,
        x="employment_income",
        y="change_in_net_income",
        animation_frame="year",
        color_discrete_sequence=[BLUE_PRIMARY],
        title=f"Change in Net Income - {reform_name}<br>Single Elderly Filer with $30,000 in Social Security Benefits",
        labels={
            "employment_income": "Employment Income ($)",
            "change_in_net_income": "Change in Net Income ($)",
            "year": "Year"
        }
    )
    
    fig.update_layout(
        font=dict(family="Roboto Serif"),
        xaxis_tickformat=",",
        yaxis_tickformat=",",
        font_color=BLACK,
        margin={"l": 50, "r": 50, "b": 100, "t": 100, "pad": 4},
        annotations=[
            {
                "x": 1,
                "y": -0.25,
                "xref": "paper",
                "yref": "paper",
                "text": "Source: PolicyEngine US",
                "showarrow": False,
                "font": {"family": "Roboto Serif", "size": 10, "color": DARK_GRAY},
            }
        ],
    )
    
    # Update animation settings
    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 500
    fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 300
    
    return fig

# Function to create animated comparison graph
def create_animated_comparison_graph(df, reform_name):
    """Create an animated graph comparing reform and baseline net income across years"""
    
    # Reshape data for comparison plot
    df_melted = pd.melt(
        df,
        id_vars=["employment_income", "year"],
        value_vars=["reform_net_income", "baseline_net_income"],
        var_name="Scenario",
        value_name="net_income"
    )
    
    # Clean up scenario names
    df_melted['Scenario'] = df_melted['Scenario'].replace({
        'reform_net_income': 'Reform',
        'baseline_net_income': 'Baseline'
    })
    
    fig = px.line(
        df_melted,
        x="employment_income",
        y="net_income",
        color="Scenario",
        animation_frame="year",
        color_discrete_map={
            "Reform": BLUE_PRIMARY,
            "Baseline": DARK_GRAY
        },
        title=f"Net Income Comparison: Reform vs Baseline - {reform_name}<br>Single Elderly Filer with $30,000 in Social Security Benefits",
        labels={
            "employment_income": "Employment Income ($)",
            "net_income": "Net Income ($)",
            "year": "Year"
        }
    )
    
    fig.update_layout(
        font=dict(family="Roboto Serif"),
        xaxis_tickformat=",",
        yaxis_tickformat=",",
        font_color=BLACK,
        margin={"l": 50, "r": 50, "b": 100, "t": 100, "pad": 4},
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        ),
        annotations=[
            {
                "x": 1,
                "y": -0.25,
                "xref": "paper",
                "yref": "paper",
                "text": "Source: PolicyEngine US",
                "showarrow": False,
                "font": {"family": "Roboto Serif", "size": 10, "color": DARK_GRAY},
            }
        ],
    )
    
    # Update animation settings
    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 500
    fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 300
    
    return fig

print("Multi-year calculation and animated graph functions defined!")

Multi-year calculation and animated graph functions defined!


In [5]:
# Generate multi-year data for all reforms and save to CSV
print("Generating multi-year household impact data for all reforms...")
print("This may take a few minutes...\n")

# Define all reforms with their functions
reforms = {
    "Option 1: Full Repeal": get_option1_reform,
    "Option 2: 85% Taxation": get_option2_reform,
    "Option 3: 85% with Senior Deduction": get_option3_reform,
    "Option 4: $500 Tax Credit": get_option4_reform,
    "Option 5: Roth-Style Swap": get_option5_reform,
    "Option 6: Phased Roth-Style": get_option6_reform,
    "Option 7: Eliminate Senior Deduction": get_option7_reform
}

# Calculate data for all reforms
all_reforms_data = []
for reform_name, reform_func in reforms.items():
    print(f"Processing {reform_name}...")
    df = calculate_all_years_impact(reform_func, reform_name)
    all_reforms_data.append(df)
    print(f"  Completed {reform_name}\n")

# Combine all dataframes
combined_df = pd.concat(all_reforms_data, ignore_index=True)

# Save to CSV
csv_filename = "household_impacts_all_years.csv"
combined_df.to_csv(csv_filename, index=False)
print(f"Data saved to {csv_filename}")
print(f"Total rows: {len(combined_df):,}")
print(f"Columns: {', '.join(combined_df.columns)}")

Generating multi-year household impact data for all reforms...
This may take a few minutes...

Processing Option 1: Full Repeal...
  Calculating Option 1: Full Repeal for 2026...
  Calculating Option 1: Full Repeal for 2027...
  Calculating Option 1: Full Repeal for 2028...
  Calculating Option 1: Full Repeal for 2029...
  Calculating Option 1: Full Repeal for 2030...
  Calculating Option 1: Full Repeal for 2031...
  Calculating Option 1: Full Repeal for 2032...
  Calculating Option 1: Full Repeal for 2033...
  Calculating Option 1: Full Repeal for 2034...
  Calculating Option 1: Full Repeal for 2035...
  Completed Option 1: Full Repeal

Processing Option 2: 85% Taxation...
  Calculating Option 2: 85% Taxation for 2026...
  Calculating Option 2: 85% Taxation for 2027...
  Calculating Option 2: 85% Taxation for 2028...
  Calculating Option 2: 85% Taxation for 2029...
  Calculating Option 2: 85% Taxation for 2030...
  Calculating Option 2: 85% Taxation for 2031...
  Calculating Option 2:

## Option 1: Full Repeal of Social Security Benefits Taxation

This option completely eliminates federal income taxation of Social Security benefits, returning to the pre-1984 policy where benefits were not subject to income tax.

In [7]:
# Option 1: Full Repeal - Animated Multi-Year Analysis
print("Creating animated graphs for Option 1: Full Repeal...")

# Filter data for Option 1
option1_df = combined_df[combined_df['reform'] == "Option 1: Full Repeal"].copy()

# Create and display animated change in net income graph
fig1_change = create_animated_change_graph(option1_df, "Option 1: Full Repeal")
fig1_change.show()

# Create and display animated comparison graph
fig1_comparison = create_animated_comparison_graph(option1_df, "Option 1: Full Repeal")
fig1_comparison.show()

print("Option 1 animated graphs created!")

Creating animated graphs for Option 1: Full Repeal...


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## Option 2: Taxation of 85% of Social Security Benefits

This option taxes 85% of Social Security benefits for all recipients, regardless of income level, eliminating the current threshold system.

In [ ]:
# Option 2: 85% Taxation - Animated Multi-Year Analysis
print("Creating animated graphs for Option 2: 85% Taxation...")

# Filter data for Option 2
option2_df = combined_df[combined_df['reform'] == "Option 2: 85% Taxation"].copy()

# Create and display animated change in net income graph
fig2_change = create_animated_change_graph(option2_df, "Option 2: 85% Taxation")
fig2_change.show()

# Create and display animated comparison graph
fig2_comparison = create_animated_comparison_graph(option2_df, "Option 2: 85% Taxation")
fig2_comparison.show()

print("Option 2 animated graphs created!")

## Option 3: 85% Taxation with Permanent Senior Deduction Extension

This option combines taxation of 85% of benefits with a permanent extension of the senior deduction that would otherwise expire in 2028.

In [ ]:
# Option 3: 85% with Senior Deduction - Animated Multi-Year Analysis
print("Creating animated graphs for Option 3: 85% with Senior Deduction...")

# Filter data for Option 3
option3_df = combined_df[combined_df['reform'] == "Option 3: 85% with Senior Deduction"].copy()

# Create and display animated change in net income graph
fig3_change = create_animated_change_graph(option3_df, "Option 3: 85% with Senior Deduction")
fig3_change.show()

# Create and display animated comparison graph
fig3_comparison = create_animated_comparison_graph(option3_df, "Option 3: 85% with Senior Deduction")
fig3_comparison.show()

print("Option 3 animated graphs created!")

## Option 4: Social Security Tax Credit System ($500 Credit)

This option replaces the senior deduction with a $500 nonrefundable tax credit while taxing 85% of benefits.

In [ ]:
# Option 4: Tax Credit System - Animated Multi-Year Analysis
print("Creating animated graphs for Option 4: $500 Tax Credit...")

# Filter data for Option 4
option4_df = combined_df[combined_df['reform'] == "Option 4: $500 Tax Credit"].copy()

# Create and display animated change in net income graph
fig4_change = create_animated_change_graph(option4_df, "Option 4: $500 Tax Credit")
fig4_change.show()

# Create and display animated comparison graph
fig4_comparison = create_animated_comparison_graph(option4_df, "Option 4: $500 Tax Credit")
fig4_comparison.show()

print("Option 4 animated graphs created!")

## Option 5: Roth-Style Swap

This option eliminates Social Security benefit taxation while making employer payroll contributions taxable income.

In [ ]:
# Option 5: Roth-Style Swap - Animated Multi-Year Analysis
print("Creating animated graphs for Option 5: Roth-Style Swap...")

# Filter data for Option 5
option5_df = combined_df[combined_df['reform'] == "Option 5: Roth-Style Swap"].copy()

# Create and display animated change in net income graph
fig5_change = create_animated_change_graph(option5_df, "Option 5: Roth-Style Swap")
fig5_change.show()

# Create and display animated comparison graph
fig5_comparison = create_animated_comparison_graph(option5_df, "Option 5: Roth-Style Swap")
fig5_comparison.show()

print("Option 5 animated graphs created!")

## Option 6: Phased Roth-Style Swap

This option implements a gradual transition to the Roth-style system over multiple years, phasing in employer contribution taxation while phasing out benefit taxation.

In [ ]:
# Option 6: Phased Roth-Style - Animated Multi-Year Analysis
print("Creating animated graphs for Option 6: Phased Roth-Style...")

# Filter data for Option 6
option6_df = combined_df[combined_df['reform'] == "Option 6: Phased Roth-Style"].copy()

# Create and display animated change in net income graph
fig6_change = create_animated_change_graph(option6_df, "Option 6: Phased Roth-Style")
fig6_change.show()

# Create and display animated comparison graph
fig6_comparison = create_animated_comparison_graph(option6_df, "Option 6: Phased Roth-Style")
fig6_comparison.show()

print("Option 6 animated graphs created!")

## Option 7: Eliminate Bonus Senior Deduction

This option eliminates the $6,000 bonus senior deduction from the One Big Beautiful Bill that includes a 6% phase-out beginning at $75k for single filers and $150k for joint filers. Since this deduction expires in 2029, Option 7 only has revenue impact from 2026-2028.

In [ ]:
# Option 7: Eliminate Senior Deduction - Animated Multi-Year Analysis
print("Creating animated graphs for Option 7: Eliminate Senior Deduction...")

# Filter data for Option 7
option7_df = combined_df[combined_df['reform'] == "Option 7: Eliminate Senior Deduction"].copy()

# Create and display animated change in net income graph
fig7_change = create_animated_change_graph(option7_df, "Option 7: Eliminate Senior Deduction")
fig7_change.show()

# Create and display animated comparison graph
fig7_comparison = create_animated_comparison_graph(option7_df, "Option 7: Eliminate Senior Deduction")
fig7_comparison.show()

print("Option 7 animated graphs created!")